In [1]:
%matplotlib notebook
import requests
import json
import time
import pathlib
import os
import matplotlib.pyplot as plt
import h5py


def dict_to_string_recursive(d):
    lines=[]
    for k,v in d.items():
        if isinstance(v, dict):
            lines.append(k + ':')
            lines.extend(['  ' + l for l in dict_to_string_recursive(v)])
        else:
            lines.append(k + ': ' + str(v))
    return lines

In [2]:
server='http://localhost:8080'
requests.get(server)
dashboard = requests.get(server + '/dashboard')
dash=json.loads(dashboard.text)


print('\n'.join(dict_to_string_recursive(dash)))

Measurement:
  PixelEventRate: 459227
  ElapsedTime: 61.727
  StartDateTime: 1733528535989
  FrameCount: 62
  TimeLeft: 0.0
  Status: DA_IDLE
Detector:
  DetectorType: Tpx3
Server:
  Notifications: []
  SoftwareVersion: 2.1.4
  SoftwareTimestamp: 2021/03/31 13:40


In [3]:
bpc_file=r"C:\SoPhy\pixelconfig_20240514.bpc"
dacs_file=r"C:\SoPhy\pixelconfig_20240514.bpc.dacs"

frame_time=1
n_frames=99999

resp= requests.get(server + '/config/load?format=pixelconfig&file=' + bpc_file)
print(resp.text)
resp= requests.get(server + '/config/load?format=dacs&file=' + dacs_file)
print(resp.text)

config=json.loads(requests.get(server + '/detector/config').text)
print('\n'.join(dict_to_string_recursive(config)))
config['BiasVoltage']=100
config['TriggerMode']='CONTINUOUS'
config['ExposureTime']=config['TriggerPeriod']=frame_time
config['nTriggers']=n_frames
resp=requests.put(server + '/detector/config', data=json.dumps(config))
print(resp.text)

Successfully uploaded pixel config.

Successfully uploaded DAC settings.

Fan1PWM: 70
Fan2PWM: 50
BiasVoltage: 100
BiasEnabled: True
ChainMode: NONE
TriggerIn: 0
TriggerOut: 0
Polarity: Positive
TriggerMode: CONTINUOUS
ExposureTime: 1.0
TriggerPeriod: 1.0
nTriggers: 99999
DetectorOrientation: UP
PeriphClk80: False
TriggerDelay: 0.0
Tdc: ['PN0123', 'PN0123']
LogLevel: 1
Successfully uploaded detector config.


In [12]:
dest='C:/serval_test'
destination = {
    "Raw": [{"Base": pathlib.Path(dest).as_uri(),
             "FilePattern": "raw%Hms_",
             }],
}

dest2={
    "Raw": [{
        # "Base": f"tcp://127.0.0.1:1234",
        "FilePattern": "",
    }],

    "Preview": {
        "Period": 0.1,
        "SamplingMode": "skipOnFrame",
        "ImageChannels": [{
            "Base": f"http://localhost:8082",
            "Format": "png",
            "Mode": "count",
            # }, {
            #     "Base": f"tcp://{self.preview_ip_total[0]}:{self.preview_ip_total[1]}",
            #     "Format": "jsonimage",
            #     "Mode": "count",
            #     "IntegrationSize": -1,
            #     "IntegrationMode": "last"
        }]
    }
}
print(dest2)
destination['Preview']=dest2['Preview']
print(destination)
resp=requests.put(server + '/server/destination', data=json.dumps(destination))
print(resp.text)
resp=requests.get(server + '/server/destination')
print(resp.text)

{'Raw': [{'FilePattern': ''}], 'Preview': {'Period': 0.1, 'SamplingMode': 'skipOnFrame', 'ImageChannels': [{'Base': 'http://localhost:8082', 'Format': 'png', 'Mode': 'count'}]}}
{'Raw': [{'Base': 'file:///C:/serval_test', 'FilePattern': 'raw%Hms_'}], 'Preview': {'Period': 0.1, 'SamplingMode': 'skipOnFrame', 'ImageChannels': [{'Base': 'http://localhost:8082', 'Format': 'png', 'Mode': 'count'}]}}
Successfully uploaded destination configuration.

{
  "Raw" : [ {
    "Base" : "file:///C:/serval_test",
    "FilePattern" : "raw%Hms_"
  } ],
  "Preview" : {
    "Period" : 0.1,
    "SamplingMode" : "skipOnFrame",
    "ImageChannels" : [ {
      "Base" : "http://localhost:8082",
      "Format" : "png",
      "Mode" : "count",
      "IntegrationSize" : 0
    } ]
  }
}


In [13]:
resp=requests.get(server+'/measurement/start')
print(resp.text)

Successfully started measurement.


In [11]:
resp=requests.get(server+'/measurement/stop')
print(resp.text)

Successfully stopped measurement.


In [ ]:
for file in os.listdir(dest):
    if file.endswith('.tpx3'):
        print(file)

In [ ]:
for file in os.listdir(dest):
    if file.endswith('.h5'):
        with h5py.File(dest + '/' + file, 'r') as f:
            x=f['x'][:]
            y=f['y'][:]
        plt.figure()
        plt.hist2d(x, y, bins=(256,256), range=((0,256),(0,256)), cmap='jet')
        plt.colorbar()